In [ ]:
import netCDF4 as nc
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from mpl_toolkits.axes_grid1 import ImageGrid
from matplotlib.animation import FuncAnimation

import os
import glob
import pandas as pd
import json
from dotenv import load_dotenv
from functools import partial

from utils.db_tools import get_data, get_db

In [3]:
model = "bruss"
run_id = "abd_test"
load_dotenv()
data_dir = os.getenv("DATA_DIR")
output_dir = os.getenv("OUT_DIR")
output_dir = os.path.join(output_dir, model, run_id)
os.makedirs(output_dir, exist_ok=True)
df0 = get_db(os.path.join(data_dir, model, run_id))
df0['run_id'].unique()

array(['abd_test'], dtype=object)

In [4]:
len(df0)

378

In [4]:
df = df0.copy()
df = df[df['run_id'] == run_id]

In [ ]:
def filter_blowup(df, threshold=1e3):
    valid_rows = []
    for i, row in df.iterrows():
        data = get_data(row)
        if np.isfinite(data).all() and np.max(data) < threshold:
            valid_rows.append(row)
    return pd.DataFrame(valid_rows)

df = filter_blowup(df)
len(df)

In [ ]:
df[(df['A'] == 0.1) & (df['B'] == 0.125) & (df['Du'] == 1.0)].iloc[0]['filename']

In [ ]:
def ab_grid(
    df, component_idx=0, frame=-1, sigdigits=2, var1="A", var2="B", file="", an=False
):
    if len(df) == 0:
        return None

    df = df.sort_values(by=[var1, var2])
    A_count = len(df[var1].unique())
    print(A_count)
    B_count = int(len(df) / A_count)
    print(B_count)
    fig = plt.figure(figsize=(15, 12))
    grid = ImageGrid(fig, 111, nrows_ncols=(A_count, B_count), axes_pad=(0.1, 0.3))

    ims = []

    for i, row in df.iterrows():
        data = get_data(row)
        f_min = data.min()
        f_max = data.max()
        ims.append((row, data[0, frame, :, component_idx::2], f_min, f_max))
        
    for ax, (row, im, f_min, f_max) in zip(grid, ims):
        ax.set_title(
            f"{var1}={row[var1]:.{sigdigits}f}\n{var2} = {row[var2]:.{sigdigits}f}",
            fontsize=6,
        )
        ax.imshow(im, cmap="viridis", vmin=f_min, vmax=f_max)
        ax.set_aspect("equal")
        ax.axis("off")
        
    row = df.iloc[0]
    if frame == -1:
        time = row["dt"] * row["Nt"]
    else:
        time = row["dt"] * frame * row["Nt"] / row["n_snapshots"]
    fig.suptitle(
        f"{row['model'].capitalize()}, Nx={row['Nx']}, dx={row['dx']}, dt={row['dt']}, T={time:.2f}",
        fontsize=16,
    )

    if file != "":
        plt.savefig(file, dpi=100)
    # if not an:
    #     plt.show()
    return grid


In [ ]:
ab_grid(df, 0, var1='A', var2='B')

In [12]:
def plot(data, global_min, global_max):
    fig, axes = plt.subplots(1, 2, figsize=(12, 6), gridspec_kw={"wspace": 0.4})
    ims = []
    for coupled_idx, ax in enumerate(axes):
        matrix = data[0, 0, :, coupled_idx::2]
        matrix /= np.max(matrix)
        im = ax.imshow(matrix, cmap="viridis", aspect="equal", vmin=0, vmax=1)
        ax.set_title(f"Snapshot 1, Component {coupled_idx + 1}")
        ims.append(im)
    return fig, axes, ims

def animate(snapshot, data, ims, axes):
    for coupled_idx, (ax, im) in enumerate(zip(axes, ims)):
        matrix = data[0, snapshot, :, coupled_idx::2]
        matrix /= matrix.max()  # Normalize
        im.set_array(matrix)
        name = "u" if coupled_idx == 0 else "v"
        ax.set_title(f"Snapshot {snapshot + 1}, {name}")
    return ims

def make_animation(data, name, out_dir):
    global_min = np.min(data)
    global_max = np.max(data)
    fig, axes, ims = plot(data, global_min, global_max)
    ani = animation.FuncAnimation(
        fig,
        partial(animate, data=data, ims=ims, axes=axes),
        frames=data.shape[1],
        interval=100,
        blit=True,
    )
    out_name = os.path.join(out_dir, f"{name}_output_norm.gif")
    ani.save(out_name, writer="ffmpeg", dpi=150)
    plt.close(fig)

In [7]:
# def plot(data, coupled_idx=0):
#     global_min = np.min(data)
#     global_max = np.max(data)
#     fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(12, 6))

#     # Display the first snapshot initially; this will be updated in the animation
#     matrix = data[0, 0, :, coupled_idx::2]
#     im = ax.imshow(
#         matrix, cmap="viridis", aspect="equal", vmin=glo, vmax=1
#     )
#     return fig, ax, im


# def animate(snapshot, coupled_idx, data, im, ax):
#     print(f"Animate called with idx {snapshot}")
#     matrix = data[0, snapshot, :, coupled_idx::2]
#     matrix /= matrix.max()
#     im.set_array(matrix)  # Update data for each coupled component
#     name = "u" if coupled_idx == 0 else "v"
#     ax.set_title(
#         f"Snapshot {snapshot + 1}, {name}"
#     )
#     return [im]


# def make_animation(data, name, out_dir, coupled_idx):
#     fig, ax, im = plot(data, coupled_idx)
#     print(data.shape[1])
#     ani = animation.FuncAnimation(
#         fig,
#         partial(animate, coupled_idx=coupled_idx, data=data, im=im, ax=ax),
#         frames=data.shape[1],
#         interval=100,
#         blit=True,
#     )
#     out_name = os.path.join(out_dir, f"{name}_output.gif")
#     ani.save(out_name, writer="ffmpeg", dpi=150)
#     plt.close(fig)

In [13]:
Du = 1
Dv = 4
A = 5
for B_mult in [1.25, 1.75, 2, 2.5, 3, 4, 5]:
    B=A*B_mult
    df_filt = df[(df['Du'] == Du) & (df['Dv'] == Dv) & (df['A'] == A) & (df['B'] == B)]
    row = df_filt.iloc[0]
    data = get_data(row)
    make_animation(data, f"Du_{Du}-Dv_{Dv}-A_{A}-B_{B}_norm", "")

MovieWriter ffmpeg unavailable; using Pillow instead.
MovieWriter ffmpeg unavailable; using Pillow instead.
MovieWriter ffmpeg unavailable; using Pillow instead.
MovieWriter ffmpeg unavailable; using Pillow instead.
MovieWriter ffmpeg unavailable; using Pillow instead.
MovieWriter ffmpeg unavailable; using Pillow instead.
MovieWriter ffmpeg unavailable; using Pillow instead.


In [ ]:
for Du in [1., 2., 3.]:
    for Dv_ratio in [4, 11, 18]:
        Dv = Dv_ratio * Du
        df_filt = df[(df['Du'] == Du) & (df['Dv'] == Dv)]
        # ab_grid(df_filt, 0, frame=-1, sigdigits=1, var1="A", var2="B", file=f"../out/abd_grids/Du_{Du}-Dv_{Dv}.png")
        convergence_plot(df_filt, sigdigits=1, var1="A", var2="B", file="../out/abd_grids/a_b_dt.png")

In [ ]:
def convergence_plot(df, sigdigits=2, var1="A", var2="B", file="", mode="dev"):
    if mode == "dev":
        text = "Deviation ||u(t) - u*||"
    elif mode == "dx":
        text = "Spatial Derivative ||∇u(t)||"
    elif mode == "dt":
        text = "Time Derivative ||du/dt||"
    else:
        raise ValueError("mode must be 'dev', 'dx', or 'dt'")
    
    if len(df) == 0:
        return None
    start_frame = 10
    df = df.sort_values(by=[var1, var2])
    df = df.reset_index(drop=True)
    A_count = len(df[var1].unique())
    B_count = int(len(df) / A_count)
    fig, axes = plt.subplots(A_count, B_count, figsize=(3 * B_count + 1, 5 * A_count))
    axes = np.atleast_2d(axes)
    for i, row in df.iterrows():
        data = get_data(row)
        steady_state = np.zeros_like(data[0, 0, :, :])
        steady_state[:, 0::2] = row["A"]
        steady_state[:, 1::2] = row["B"] / row["A"]
        deviations = []
        time_derivatives = []
        spatial_derivatives = []
        du_dt = np.gradient(data[0, :, :, 0::2], row["dt"], axis=0)
        for j in range(start_frame, data.shape[1]):
            u = data[0, j, :, 0::2]
            v = data[0, j, :, 1::2]
            du_dx = np.gradient(u, row["dx"], axis=0)
            dv_dx = np.gradient(v, row["dx"], axis=0)
            deviations.append(np.linalg.norm(data[0, j, :, :] - steady_state))
            time_derivatives.append(np.linalg.norm(du_dt[j]))
            spatial_derivatives.append(np.linalg.norm(du_dx) + np.linalg.norm(dv_dx))

        if mode == "dev":
            values = deviations
        elif mode == "dx":
            values = spatial_derivatives
        elif mode == "dt":
            values = time_derivatives
        axes[i // B_count, i % B_count].plot(
            np.arange(0, data.shape[1]-start_frame) * row["dt"] * row["Nt"] / row["n_snapshots"],
            values,
        )
        axes[i // B_count, i % B_count].set_title(
            f"{var1}={row[var1]:.{sigdigits}f}\n{var2} = {row[var2]:.{sigdigits}f}",
            fontsize=6,
        )
        # axes[i // B_count, i % B_count].axis("off")
    row = df.iloc[0]
    time = row["dt"] * row["Nt"]
    fig.suptitle(
        f"{row['model'].capitalize()}, Nx={row['Nx']}, dx={row['dx']}, dt={row['dt']}, T={time:.2f}, {text}",
        fontsize=4*B_count,
    )
    plt.tight_layout()
    plt.subplots_adjust(top=0.95)
    # plt.show()
    if file != "":
        fig.savefig(file, dpi=100)
    plt.close()

In [ ]:
for Du in [1.0, 2.0, 3.0]:
    for Dv_ratio in [4, 11, 18]:
        Dv = Dv_ratio * Du
        df_filt = df[(df["Du"] == Du) & (df["Dv"] == Dv)]
        # ab_grid(df_filt, 0, frame=-1, sigdigits=1, var1="A", var2="B", file=f"../out/abd_grids/final_Du_{Du}-Dv_{Dv}.png")
        dir = "../out/abd_burn_in/"
        convergence_plot(
            df_filt,
            sigdigits=1,
            var1="A",
            var2="B",
            file=dir + f"dev_Du_{Du}-Dv_{Dv}.png",
            mode="dev",
        )
        convergence_plot(
            df_filt,
            sigdigits=1,
            var1="A",
            var2="B",
            file=dir + f"dx_Du_{Du}-Dv_{Dv}.png",
            mode="dx",
        )
        convergence_plot(
            df_filt,
            sigdigits=1,
            var1="A",
            var2="B",
            file=dir + f"dt_Du_{Du}-Dv_{Dv}.png",
            mode="dt",
        )
